In [ ]:
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-l19zpw5a
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-l19zpw5a
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 46705844b39ededc0fcef1de90e73923480a6446
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import networkx as nx
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import from_networkx
from torch_geometric.data import HeteroData
from torch_geometric.nn import SAGEConv
from tqdm import tqdm

In [ ]:
# Load the tabular dataset
df = pd.read_csv("credit_card_transactions-ibm_v2.csv").sample(n=100000, random_state=42)
df["card_id"] = df["User"].astype(str) + "_" + df["Card"].astype(str)
df["Amount"]=df["Amount"].str.replace("$","").astype(float)
df["Hour"] = df["Time"].str [0:2]
df["Minute"] = df["Time"].str [3:5]
df = df.drop(["Time","User","Card"],axis=1)
df["Errors?"]= df["Errors?"].fillna("No error")
df = df.drop(columns=["Merchant State","Zip"],axis=1)
df["Is Fraud?"] = df["Is Fraud?"].apply(lambda x: 1 if x == 'Yes' else 0)
df["Merchant City"]=LabelEncoder().fit_transform(df["Merchant City"])
df["Use Chip"]=LabelEncoder().fit_transform(df["Use Chip"])
df["Errors?"]=LabelEncoder().fit_transform(df["Errors?"])

In [ ]:
# fraudulent = df[df["Is Fraud?"] == 1]
# non_fraudulent = df[df["Is Fraud?"] == 0]

# print(f"Number of fraudulent transactions: {len(fraudulent)}")
# print(f"Number of non-fraudulent transactions: {len(non_fraudulent)}")

# desired_ratio = 1  # Ratio of non-fraud to fraud (e.g., 1 means 1:1 balance)
# non_fraud_sample_size = len(fraudulent) * desired_ratio

# non_fraudulent_sampled = non_fraudulent.sample(n=non_fraud_sample_size, random_state=42)

# balanced_df = pd.concat([fraudulent, non_fraudulent_sampled])
# balanced_df = balanced_df.sample(frac=1, random_state=42)

# print(f"Balanced dataset size: {len(balanced_df)}")
# print(balanced_df["Is Fraud?"].value_counts())
# df = balanced_df.copy()

In [ ]:
class GraphConstruction:
    def __init__(self, nodes, edges, features=None):
        self.g_nx = nx.DiGraph()
        self.add_nodes(nodes)
        self.add_edges(edges)
        self.node_features = features if features is not None else {}

    def add_nodes(self, nodes):
        for ntype, nodelist in nodes.items():
            for node in nodelist:
                self.g_nx.add_node(node, ntype=ntype)

    def add_edges(self, edges):
        for edge in edges:
            self.g_nx.add_edges_from(edge)

    def to_pyg_data(self):
        data = HeteroData()
        node_id_mapping = {ntype: [] for ntype in set(nx.get_node_attributes(self.g_nx, 'ntype').values())}

        print("Processing nodes...")
        for node, ndata in tqdm(self.g_nx.nodes(data=True), desc="Nodes"):
            ntype = ndata['ntype']
            if 'x' not in data[ntype]:
                data[ntype].x = []
            node_id_mapping[ntype].append(node)
            if self.node_features and ntype in self.node_features:
                if node in self.node_features[ntype].index:
                    features = self.node_features[ntype].loc[node].values
                    features = [float(f) if isinstance(f, (int, float)) else 0.0 for f in features]
                    data[ntype].x.append(features)
                else:
                    data[ntype].x.append([0.0] * self.node_features[ntype].shape[1])

        print("Converting node features to tensors...")
        for ntype in tqdm(data.node_types, desc="Node Types"):
            data[ntype].x = torch.tensor(data[ntype].x, dtype=torch.float)

        print("Processing edges...")
        for u, v in tqdm(self.g_nx.edges(), desc="Edges"):
            u_type = self.g_nx.nodes[u]['ntype']
            v_type = self.g_nx.nodes[v]['ntype']
            edge_type = (u_type, 'to', v_type)
            if edge_type not in data.edge_types:
                data[edge_type].edge_index = [[], []]
            if u in node_id_mapping[u_type] and v in node_id_mapping[v_type]:
                u_index = node_id_mapping[u_type].index(u)
                v_index = node_id_mapping[v_type].index(v)
                data[edge_type].edge_index[0].append(u_index)
                data[edge_type].edge_index[1].append(v_index)

        print("Converting edge indices to tensors...")
        for edge_type in tqdm(data.edge_types, desc="Edge Types"):
            edge_index = data[edge_type].edge_index
            data[edge_type].edge_index = torch.tensor(edge_index, dtype=torch.long)

        return data

In [ ]:
class GraphSAGE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers):
        super(GraphSAGE, self).__init__()
        self.convs = torch.nn.ModuleList()
        self.convs.append(SAGEConv(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(SAGEConv(hidden_channels, hidden_channels))
        self.convs.append(SAGEConv(hidden_channels, out_channels))

    def forward(self, x, edge_index):
        for conv in self.convs[:-1]:
            x = conv(x, edge_index)
            x = torch.relu(x)
        x = self.convs[-1](x, edge_index)
        return torch.log_softmax(x, dim=-1)

In [ ]:
# Encode categorical features
le_card = LabelEncoder()
le_merchant = LabelEncoder()
df['card_id_enc'] = le_card.fit_transform(df['card_id'])
df['merchant_enc'] = le_merchant.fit_transform(df['Merchant Name'])

# Balancing the dataset without resetting the index
fraudulent = df[df["Is Fraud?"] == 1]
non_fraudulent = df[df["Is Fraud?"] == 0]

desired_ratio = 1  # 2:1 balance
non_fraud_sample_size = round(len(fraudulent) * desired_ratio)
non_fraudulent_sampled = non_fraudulent.sample(n=non_fraud_sample_size, random_state=42)

balanced_df = pd.concat([fraudulent, non_fraudulent_sampled])
balanced_df = balanced_df.sample(frac=1, random_state=42)  # Do not reset index
print(f"Balanced dataset size: {len(balanced_df)}")
print(balanced_df["Is Fraud?"].value_counts())

df = balanced_df.copy()
# Transform labels using the same encoders
df['card_id_enc'] = le_card.transform(df['card_id'])
df['merchant_enc'] = le_merchant.transform(df['Merchant Name'])

# Split data into training and inductive sets
cutoff = round(0.7 * len(df))
train_data = df.iloc[:cutoff]
inductive_data = df.iloc[cutoff:]

# Prepare node features
transaction_features = train_data.drop(columns=['card_id', 'Merchant Name', 'Is Fraud?', 'card_id_enc', 'merchant_enc'])
transaction_features.index = train_data.index
client_features = pd.DataFrame(index=train_data['card_id_enc'].unique())
merchant_features = pd.DataFrame(index=train_data['merchant_enc'].unique())

train_mask = torch.zeros(len(df), dtype=torch.bool)
val_mask = torch.zeros(len(df), dtype=torch.bool)

train_mask[:cutoff] = True  # Training nodes
val_mask[cutoff:] = True

# Create nodes and edges
nodes = {
    'client': df['card_id_enc'].unique(),
    'merchant': df['merchant_enc'].unique(),
    'transaction': df.index
}
edges = [
    list(zip(df['card_id_enc'], df.index)),
    list(zip(df.index, df['merchant_enc']))
]
features = {
    'transaction': df.drop(columns=['card_id', 'Merchant Name', 'Is Fraud?', 'card_id_enc', 'merchant_enc']),
    'client': pd.DataFrame(index=df['card_id_enc'].unique()),
    'merchant': pd.DataFrame(index=df['merchant_enc'].unique())
}

# Build graph
graph = GraphConstruction(nodes, edges, features)
data = graph.to_pyg_data()

data['transaction'].train_mask = train_mask
data['transaction'].val_mask = val_mask
data['transaction'].y = torch.tensor(df['Is Fraud?'].values, dtype=torch.long)

Balanced dataset size: 234
Is Fraud?
1    117
0    117
Name: count, dtype: int64
Processing nodes...


Nodes: 100%|██████████| 602/602 [00:00<00:00, 9138.48it/s]


Converting node features to tensors...


Node Types: 100%|██████████| 3/3 [00:00<00:00, 348.34it/s]


Processing edges...


Edges: 100%|██████████| 468/468 [00:00<00:00, 25593.36it/s]


Converting edge indices to tensors...


Edge Types: 100%|██████████| 3/3 [00:00<00:00, 4550.78it/s]


In [ ]:
# Define model
in_channels = transaction_features.shape[1]
hidden_channels = 64
out_channels = 2  # Binary classification
num_layers = 2
model = GraphSAGE(in_channels, hidden_channels, out_channels, num_layers)

# Training setup
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()
# Training loop
model.train()
target_labels = torch.tensor(train_data['Is Fraud?'].values, dtype=torch.long)  # Convert to tensor

for epoch in range(201):
    optimizer.zero_grad()

    out = model(data['transaction'].x, data[('transaction', 'to', 'merchant')].edge_index)
    loss = criterion(out[data['transaction'].train_mask], data['transaction'].y[data['transaction'].train_mask])
    loss.backward()
    optimizer.step()
    if epoch % 10 == 0:
        print(f'Epoch {epoch+1}, Loss: {loss.item()}')

Epoch 1, Loss: 7.1475043296813965
Epoch 11, Loss: 1.1132194995880127
Epoch 21, Loss: 0.7265760898590088
Epoch 31, Loss: 0.7345401048660278
Epoch 41, Loss: 0.6320815086364746
Epoch 51, Loss: 0.6047638058662415
Epoch 61, Loss: 0.5975351333618164
Epoch 71, Loss: 0.5924930572509766
Epoch 81, Loss: 0.572880208492279
Epoch 91, Loss: 0.6029146313667297
Epoch 101, Loss: 0.6359283328056335
Epoch 111, Loss: 0.8658685684204102
Epoch 121, Loss: 0.6251945495605469
Epoch 131, Loss: 0.58433598279953
Epoch 141, Loss: 0.5790935754776001
Epoch 151, Loss: 0.5592132210731506
Epoch 161, Loss: 0.5712076425552368
Epoch 171, Loss: 0.5983248353004456
Epoch 181, Loss: 0.5569855570793152
Epoch 191, Loss: 0.5679579377174377
Epoch 201, Loss: 0.5701553821563721


In [ ]:
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

model.eval()
with torch.no_grad():
    val_logits = out[data['transaction'].val_mask]
    val_preds = torch.argmax(val_logits, dim=1)
    val_labels = data['transaction'].y[data['transaction'].val_mask]
print(val_labels.bincount())
f1 = f1_score(val_labels.cpu(), val_preds.cpu(), average='binary')
accuracy = accuracy_score(val_labels.cpu(), val_preds.cpu())
conf_matrix = confusion_matrix(val_labels.cpu(), val_preds.cpu())

print(f"Validation F1 Score: {f1:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

tensor([34, 36])
Validation F1 Score: 0.7042
Validation Accuracy: 0.7000
Confusion Matrix:
[[24 10]
 [11 25]]
